## SVM- restricted

In [1]:
import pandas as pd
import numpy as np
from sklearn import cluster, covariance, manifold
import numpy as np
from sklearn.metrics import r2_score
import math
import matplotlib.pyplot as plt
import time
import os

In [2]:
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.optimize import minimize
from sklearn.feature_selection import RFECV

In [3]:
import optuna

In [4]:
# import warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error

In [6]:
path = r'C:\Users\User\Documents\Atividades_andamento\Nayana\corrige_codigos'
os.chdir(path)

In [7]:
#df =pd.read_excel('custos_versao_jun_17_2023.xlsx')
df =pd.read_excel("custos_versao_mar_14B_2024.xlsx")
df = df.drop(['Local','Barragens'],axis=1)
names = df.columns

In [8]:
df.shape

(33, 78)

In [9]:
word ='Custo_barragem' 
word = 'Dams'
X = df.drop([word],axis=1)
y = df[word]
y = np.log10(y)

escalor =  StandardScaler() # escalor = MinMaxScaler()
y_norm = escalor.fit_transform(y.values.reshape(-1,1))

## RF



In [10]:
restricted =['ReinfEmbank','GroundwaQMon', 
             'ErosProcMain', 'Volume', 
             'Area', 'Height']

In [11]:
X_restricted = df[restricted]

x_scaler = StandardScaler() # x_scaler = MinMaxScaler()
y_scaler = StandardScaler()# y_scaler = MinMaxScaler()

X_norm = x_scaler.fit_transform(X_restricted)
y_norm = y_scaler.fit_transform(y.values.reshape(-1,1))

In [12]:
X,y = X_norm,y_norm

In [13]:
def mine_test(set_parameters, X=X, y=y): # testa o ajuste
  rows=X.shape[0]
  cols =X.shape[1]

  y_pred   = []

  for i in range(rows):
    X_train = np.delete(X,i,axis=0)
    y_train = np.delete(y,i,axis=0)
    X_test  = X[i,:].reshape(1,cols)
    y_test  = y[i]
    model =  RandomForestRegressor(**set_parameters)
    #model =  SVR(**set_parameters)
    model = model.fit(X_train, y_train.reshape(-1,1))
    yi = model.predict(X_test)
    y_pred.append(yi)

  y_pred = np.array(y_pred)

  resp = mean_absolute_error(y, y_pred)
  

  return -resp


In [14]:
# Define the objective function and suggest hyperparameters values
def objective(trial):
    rf_n_estimators= [100, 200, 300]
    rf_criterion= ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
    rf_criterion= ['squared_error', 'absolute_error', 'friedman_mse']
    rf_max_depth = [None, 10, 20, 30]
    rf_min_samples_split= [2, 5, 10]
    rf_min_samples_leaf= [1, 2, 4]
    rf_max_features= [1,5,8,10, 'sqrt', 'log2']
    
    n_estimators = trial.suggest_categorical('n_estimators', rf_n_estimators)
    criterion = trial.suggest_categorical('criterion', rf_criterion)
    max_depth = trial.suggest_categorical('max_depth', rf_max_depth)
    min_samples_split = trial.suggest_categorical('min_samples_split', rf_min_samples_split)
    max_features = trial.suggest_categorical('max_features', rf_max_features)
   
    set_parameters = {'n_estimators':n_estimators,
                      'criterion': criterion,
                      'max_depth': max_depth, 
                      'min_samples_split':min_samples_split,
                      'max_features':max_features}

    adjust = mine_test( set_parameters)
    
    
    return adjust

In [15]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2024-08-22 13:05:04,456] A new study created in memory with name: no-name-cce4787d-b755-4f55-9509-ae2f97afd4ab
[I 2024-08-22 13:05:11,762] Trial 0 finished with value: -0.34188573808923733 and parameters: {'n_estimators': 100, 'criterion': 'friedman_mse', 'max_depth': 20, 'min_samples_split': 2, 'max_features': 'sqrt'}. Best is trial 0 with value: -0.34188573808923733.
[I 2024-08-22 13:05:33,376] Trial 1 finished with value: -0.4167926865431085 and parameters: {'n_estimators': 300, 'criterion': 'squared_error', 'max_depth': None, 'min_samples_split': 10, 'max_features': 'sqrt'}. Best is trial 0 with value: -0.34188573808923733.
[I 2024-08-22 13:05:56,001] Trial 2 finished with value: -0.4330679711841144 and parameters: {'n_estimators': 300, 'criterion': 'squared_error', 'max_depth': 10, 'min_samples_split': 10, 'max_features': 5}. Best is trial 0 with value: -0.34188573808923733.
[I 2024-08-22 13:06:10,658] Trial 3 finished with value: -0.43987803470083636 and parameters: {'n_estima

In [16]:
# Print the best set of hyperparameters
print('Best hyperparameters: ', study.best_params)
# Print the corresponding performance
print('Best performance: ', study.best_value)

Best hyperparameters:  {'n_estimators': 100, 'criterion': 'friedman_mse', 'max_depth': 20, 'min_samples_split': 2, 'max_features': 1}
Best performance:  -0.3187524189794635


In [18]:
best = study.best_params

n_estimators = best['n_estimators']
criterion = best['criterion'] 
max_depth = best['max_depth'] 
min_samples_split = best['min_samples_split']
max_features = best['max_features']
   
set_parameters = {'n_estimators':n_estimators,
                  'criterion': criterion,
                  'max_depth': max_depth, 
                  'min_samples_split':min_samples_split,
                  'max_features':max_features}

In [21]:
def mine_test(X,y, set_parameters= set_parameters): # testa o ajuste
  rows=X.shape[0]
  cols =X.shape[1]

  y_pred   = []

  for i in range(rows):
    X_train = np.delete(X,i,axis=0)
    y_train = np.delete(y,i,axis=0)
    X_test  = X[i,:].reshape(1,cols)
    y_test  = y[i]
    model =  RandomForestRegressor(**set_parameters)
    model = model.fit(X_train, y_train.reshape(-1,1))
    yi = model.predict(X_test)
    y_pred.append(yi)

  y_pred = np.array(y_pred)
  #print(model)

  return y_pred

In [22]:
y_pred = mine_test(X , y)
y_true = y

In [23]:
print('mae :', mean_absolute_error(y_true, y_pred))
print('rmse :', root_mean_squared_error(y_true, y_pred))
print('r-score :',r2_score(y_true, y_pred))

mae : 0.32314788152498647
rmse : 0.4561990679822124
r-score : 0.7918824103721607


In [24]:
for value in set_parameters:
    print(value,set_parameters[value])

n_estimators 100
criterion friedman_mse
max_depth 20
min_samples_split 2
max_features 1
